In [1]:
# Copy and paste the following imports at the beginning of your script
import pandas as pd
from transformers import AutoTokenizer, BertForSequenceClassification
import torch

In [2]:
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IE4483/data/processed_test_data.csv')

model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

max_length = 128
test_texts = test_data['reviews']
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')

class SentimentTestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings.input_ids)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        return item

test_dataset = SentimentTestDataset(test_encodings)

# Replace with the path to your fine-tuned model directory
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/IE4483/model")

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [3]:
# Make predictions
with torch.no_grad():
    predictions = model(**test_encodings)

predicted_labels = predictions.logits.argmax(dim=1).tolist()

test_data['predicted_sentiments'] = predicted_labels

test_data.to_csv('test_predictions.csv', index=False)